In [1]:
import pandas as pd
import folium


Raw Dataframe (행정시군구의 태양광 발전 설비 설치 용량 데이터 활용)

In [2]:
df = pd.read_csv(r"../Data/solar_power_plant.csv", encoding="EUC-KR")

df.columns = ["city", "the number of solar generator","power generation capacity(kWh)"]
df

,city,the number of solar generator,power generation capacity(kWh)
0,강원특별자치도 철원군,66,94.271395
1,강원특별자치도 양구군,12,16.922960
2,강원특별자치도 원주시,45,55.802320
3,강원특별자치도 태백시,59,52.049515
4,강원특별자치도 삼척시,71,73.293538
...,...,...,...
302,충청북도 옥천군,1339,151.809284
303,충청북도 충주시,1,0.856440
304,충청북도 제천시,1,0.663570
305,충청북도 진천군,1,0.667800


행정시군구 태양광 발전 설비 용량 데이터의 도시명(307개)과 geoJSON 행정동 데이터의 도시명(251개) 사이의 불일치 해결하기

In [ ]:
import geopandas as gpd

#GeoJSON 파일 로드
geojson_path = "..\\Data\\HangJeongDong_ver20241001.geojson"

gdf = gpd.read_file(geojson_path)
geojson_cities = set(gdf["sidonm"] + " " + gdf["sggnm"]) #GeoJSON 데이터에서의 시군구 도시명 추출

df_cities = set(df["city"]) # 시군구별 태양광 설비 용량 데이터에서의 시군구 도시명 추출


#불일치 요소 찾기

only_in_geojson = sorted(geojson_cities - df_cities)
only_in_df = sorted(df_cities - geojson_cities)

#불일치 비교 표 만들기
comparison_df = pd.DataFrame({
    "only_in_geojson": pd.Series(only_in_geojson),
    "only_in_df": pd.Series(only_in_df)
})

#comparison_df.to_csv("comparing_cities.csv")
comparison_df

In [16]:
#df 수정하기

df.loc[ df['city'] =="강원특별자치도 횡선군", "city"] ="강원특별자치도 횡성군" #시군구명 오타 수정

#세종 특별 자치시의 값들 합산하여 한 열로 통합하기

keyword = "세종특별자치시"

filtered_rows = df[df["city"].str.contains(keyword)] #세종특별자치시에 포함되는 행 필터링

new_row = {
    'city': '세종특별자치시 세종시', #GeoJSON 기준으로 맞춰주기
    'the number of solar generator': filtered_rows['the number of solar generator'].sum(),
    "power generation capacity(kWh)": filtered_rows["power generation capacity(kWh)"].sum()
}

#필터링된 행 제거
df = df[~df["city"].str.contains(keyword)].reset_index(drop=True)

#새로운 행 추가
df = pd.concat([df, pd.DataFrame([new_row])], ignore_index=True)

df

,city,the number of solar generator,power generation capacity(kWh)
0,강원특별자치도 철원군,66,94.271395
1,강원특별자치도 양구군,12,16.922960
2,강원특별자치도 원주시,45,55.802320
3,강원특별자치도 태백시,59,52.049515
4,강원특별자치도 삼척시,71,73.293538
...,...,...,...
284,충청북도 충주시,1,0.856440
285,충청북도 제천시,1,0.663570
286,충청북도 진천군,1,0.667800
287,충청북도 보은군,1,0.739260


In [17]:
#도시명 불일치 중간 점검

df_cities = set(df["city"]) # 시군구별 태양광 설비 용량 데이터에서의 시군구 도시명 추출


#불일치 요소 찾기

only_in_geojson = sorted(geojson_cities - df_cities)
only_in_df = sorted(df_cities - geojson_cities)

#불일치 비교 표 만들기
comparison_df = pd.DataFrame({
    "only_in_geojson": pd.Series(only_in_geojson),
    "only_in_df": pd.Series(only_in_df)
})

#comparison_df.to_csv("comparing_cities.csv")
comparison_df

,only_in_geojson,only_in_df
0,경기도 고양시덕양구,경기도 고양시
1,경기도 고양시일산동구,경기도 부천시
2,경기도 고양시일산서구,경기도 성남시
3,경기도 부천시소사구,경기도 수원시
4,경기도 부천시오정구,경기도 안산시
5,경기도 부천시원미구,경기도 안양시
6,경기도 성남시분당구,경기도 용인시
7,경기도 성남시수정구,경상남도 창원시
8,경기도 성남시중원구,경상북도 포항시
9,경기도 수원시권선구,전북특별자치도 전주시


In [22]:
#GeoJSON에서 시 이름 옆에 군구가 붙은 도시 폴리곤 모두 시 기준으로 통합해주기

def create_group_name(city_name): #여기서 city_name은 gdf["sggnm"]
    if '시' in city_name and city_name != "시흥시":
        return city_name.split('시')[0] + '시' #gdf["sggnm"]에 '시'가 포함되는지 여부
    return city_name

#시 기준으로 GeoJSON 데이터의 그룹 이름 생성
gdf["city_group"] = gdf["sidonm"] + " " + gdf["sggnm"].apply(create_group_name)

#그룹별 폴리곤 통합
edited_gdf = gdf.dissolve(by = 'city_group')

#통합된 폴리곤의 이름을 다시 설정
edited_gdf['city_name'] = edited_gdf.index

edited_gdf

,geometry,adm_nm,adm_cd2,sgg,sido,sidonm,sggnm,adm_cd,city_name
city_group,,,,,,,,,
강원특별자치도 강릉시,"POLYGON ((128.88276 37.59205, 128.87959 37.589...",강원특별자치도 강릉시 주문진읍,5115025000,51150,51,강원특별자치도,강릉시,32030110,강원특별자치도 강릉시
강원특별자치도 고성군,"POLYGON ((128.56537 38.27518, 128.56881 38.268...",강원특별자치도 고성군 간성읍,5182025000,51820,51,강원특별자치도,고성군,32600110,강원특별자치도 고성군
강원특별자치도 동해시,"POLYGON ((129.15502 37.47399, 129.15486 37.473...",강원특별자치도 동해시 천곡동,5117051000,51170,51,강원특별자치도,동해시,32040510,강원특별자치도 동해시
강원특별자치도 삼척시,"POLYGON ((129.08496 37.16004, 129.08486 37.160...",강원특별자치도 삼척시 도계읍,5123025000,51230,51,강원특별자치도,삼척시,32070110,강원특별자치도 삼척시
강원특별자치도 속초시,"POLYGON ((128.60753 38.1859, 128.60928 38.1836...",강원특별자치도 속초시 영랑동,5121051000,51210,51,강원특별자치도,속초시,32060510,강원특별자치도 속초시
...,...,...,...,...,...,...,...,...,...
충청북도 제천시,"POLYGON ((128.24045 36.87125, 128.23979 36.868...",충청북도 제천시 봉양읍,4315025000,43150,43,충청북도,제천시,33030110,충청북도 제천시
충청북도 증평군,"POLYGON ((127.62506 36.79211, 127.62637 36.790...",충청북도 증평군 증평읍,4374525000,43745,43,충청북도,증평군,33590140,충청북도 증평군
충청북도 진천군,"POLYGON ((127.49485 36.77758, 127.49286 36.776...",충청북도 진천군 진천읍,4375025000,43750,43,충청북도,진천군,33550110,충청북도 진천군


In [23]:
#GeoJSON 과 df 간의 도시명 불일치 중간체크

df_cities = set(df["city"]) # 시군구별 태양광 설비 용량 데이터에서의 시군구 도시명 추출

geojson_cities2 = set(edited_gdf["city_name"]) #GeoJSON 데이터에서의 시군구 도시명 추출


#불일치 요소 찾기

only_in_geojson = sorted(geojson_cities2 - df_cities)
only_in_df = sorted(df_cities - geojson_cities2)

#불일치 비교 표 만들기
comparison_df = pd.DataFrame({
    "only_in_geojson": pd.Series(only_in_geojson),
    "only_in_df": pd.Series(only_in_df)
})

#comparison_df.to_csv("comparing_cities.csv")
comparison_df #울릉도를 제외하고 모든 지역명 맞춰짐

,only_in_geojson,only_in_df
0,경상북도 울릉군,NaN


In [27]:
#df.to_csv("solar_poewer_df.csv")
#여전히 geojson에는 229개의 도시명, df에는 289개의 도시 데이터가 있다. 그러나 일치하지 않는 경우는 울릉도 뿐이다.
#즉, df에 같은 도시명을 가진 다른 행으로 된 행들 존재

#df에서 같은 지역명을 갖는 데이터끼리 합산 처리해주기
grouped_df = df.groupby('city', as_index=False).sum()

df = grouped_df
df #중복 도시를 합산 처리하니 289개에서 울릉군을 제외한 288개의 데이터가 남았다.


,city,the number of solar generator,power generation capacity(kWh)
0,강원특별자치도 강릉시,586,111.404615
1,강원특별자치도 고성군,611,126.138935
2,강원특별자치도 동해시,100,25.516230
3,강원특별자치도 삼척시,381,148.375873
4,강원특별자치도 속초시,63,5.727450
...,...,...,...
223,충청북도 제천시,679,100.151225
224,충청북도 증평군,304,34.933695
225,충청북도 진천군,756,127.497424
226,충청북도 청주시,2442,292.979795


좌표 도시별 분류 및 도시별 월 평균 합계 일사량 값 구하기

In [28]:
#지도 생성

m = folium.Map(location=[36.5, 127.5], zoom_start=7)

GeoJSON 데이터와 태양광 발전 설비 용량 데이터를 결합해 지도 위에 시각화하기

In [29]:
from branca.colormap import linear
from matplotlib.colors import Normalize, rgb2hex
from matplotlib import colormaps 

#태양광 발전 설비용량을 컬러맵으로 매핑

min_cap = df['power generation capacity(kWh)'].min()
max_cap = df['power generation capacity(kWh)'].max()
colormap = colormaps["coolwarm"]



norm = Normalize(vmin = min_cap, vmax = max_cap)
print(min_cap, max_cap) #최소, 최대 설비용량 값 확인

df["Color(hex)"] = df["power generation capacity(kWh)"].apply(lambda capacity: rgb2hex(colormap(norm(capacity)))) 
# 설비용량 값을 정규화 하여 컬러맵으로 변환

df

0.0888 1004.883525


,city,the number of solar generator,power generation capacity(kWh),Color(hex)
0,강원특별자치도 강릉시,586,111.404615,#5d7ce6
1,강원특별자치도 고성군,611,126.138935,#6282ea
2,강원특별자치도 동해시,100,25.516230,#4257c9
3,강원특별자치도 삼척시,381,148.375873,#688aef
4,강원특별자치도 속초시,63,5.727450,#3c4ec2
...,...,...,...,...
223,충청북도 제천시,679,100.151225,#5977e3
224,충청북도 증평군,304,34.933695,#445acc
225,충청북도 진천군,756,127.497424,#6282ea
226,충청북도 청주시,2442,292.979795,#9bbcff


In [ ]:
#Folium에 사용할 데이터 준비: edited_gdf데이터와 df 병합

merged = edited_gdf.merge(df, left_on="city_name", right_on="city", how = "left")
#print(aug_df.info())
print(merged.info())
merged #울릉군의 color데이터와 발전 설비용량은  null 처리 되었음을 확인할 수 있다.

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 229 entries, 0 to 228
Data columns (total 13 columns):
 #   Column                          Non-Null Count  Dtype   
---  ------                          --------------  -----   
 0   geometry                        229 non-null    geometry
 1   adm_nm                          229 non-null    object  
 2   adm_cd2                         229 non-null    object  
 3   sgg                             229 non-null    object  
 4   sido                            229 non-null    object  
 5   sidonm                          229 non-null    object  
 6   sggnm                           229 non-null    object  
 7   adm_cd                          229 non-null    object  
 8   city_name                       229 non-null    object  
 9   city                            228 non-null    object  
 10  the number of solar generator   228 non-null    float64 
 11  power generation capacity(kWh)  228 non-null    float64 
 12  Color(hex)    

,geometry,adm_nm,adm_cd2,sgg,sido,sidonm,sggnm,adm_cd,city_name,city,the number of solar generator,power generation capacity(kWh),Color(hex)
0,"POLYGON ((128.88276 37.59205, 128.87959 37.589...",강원특별자치도 강릉시 주문진읍,5115025000,51150,51,강원특별자치도,강릉시,32030110,강원특별자치도 강릉시,강원특별자치도 강릉시,586.0,111.404615,#5d7ce6
1,"POLYGON ((128.56537 38.27518, 128.56881 38.268...",강원특별자치도 고성군 간성읍,5182025000,51820,51,강원특별자치도,고성군,32600110,강원특별자치도 고성군,강원특별자치도 고성군,611.0,126.138935,#6282ea
2,"POLYGON ((129.15502 37.47399, 129.15486 37.473...",강원특별자치도 동해시 천곡동,5117051000,51170,51,강원특별자치도,동해시,32040510,강원특별자치도 동해시,강원특별자치도 동해시,100.0,25.516230,#4257c9
3,"POLYGON ((129.08496 37.16004, 129.08486 37.160...",강원특별자치도 삼척시 도계읍,5123025000,51230,51,강원특별자치도,삼척시,32070110,강원특별자치도 삼척시,강원특별자치도 삼척시,381.0,148.375873,#688aef
4,"POLYGON ((128.60753 38.1859, 128.60928 38.1836...",강원특별자치도 속초시 영랑동,5121051000,51210,51,강원특별자치도,속초시,32060510,강원특별자치도 속초시,강원특별자치도 속초시,63.0,5.727450,#3c4ec2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
224,"POLYGON ((128.24045 36.87125, 128.23979 36.868...",충청북도 제천시 봉양읍,4315025000,43150,43,충청북도,제천시,33030110,충청북도 제천시,충청북도 제천시,679.0,100.151225,#5977e3
225,"POLYGON ((127.62506 36.79211, 127.62637 36.790...",충청북도 증평군 증평읍,4374525000,43745,43,충청북도,증평군,33590140,충청북도 증평군,충청북도 증평군,304.0,34.933695,#445acc
226,"POLYGON ((127.49485 36.77758, 127.49286 36.776...",충청북도 진천군 진천읍,4375025000,43750,43,충청북도,진천군,33550110,충청북도 진천군,충청북도 진천군,756.0,127.497424,#6282ea
227,"POLYGON ((127.47171 36.47467, 127.47144 36.474...",충청북도 청주시상당구 낭성면,4311131000,43111,43,충청북도,청주시상당구,33041310,충청북도 청주시,충청북도 청주시,2442.0,292.979795,#9bbcff


In [31]:
#GeoJSON 스타일 함수 정의

def style_function(feature):
    color = feature['properties'].get('Color(hex)', None)
    if color: #색상이 있는 경우 
        return {
            "fillColor" : color,
            "color" : "black", #경계선 색
            "weight": 1, #경계선 두께
            "fillOpacity": 0.7
        }
    else: #색상 데이터가 없는 경우 투명 처리
        return {
            "fillColor" : "transparent",
            "color" : "black", #경계선 색
            "weight": 1, #경계선 두께
            "fillOpacity": 0
        }

#GeoJSON 레이어 추가
folium.GeoJson(
    merged,
    style_function = style_function,
    tooltip = folium.GeoJsonTooltip(fields=["city_name", "power generation capacity(kWh)"])
).add_to(m)

In [32]:
m.save("..\\Visualization\\solar_power_map.html")
print("지도가 'solar_power_map.html'로 저장되었습니다. 브라우저에서 확인하세요.")

지도가 'solar_power_map.html'로 저장되었습니다. 브라우저에서 확인하세요.
